In [61]:
from datetime import datetime
import pandas as pd
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots

In [2]:
# Population figures from HKSAR government
# https://www.covidvaccine.gov.hk/en/dashboard
total_population = 448800 + 808700 + 1126300 + 1142500 + 1174200 + 1071800 + 560500 + 401800
total_population

6734600

### Vaccination Figure

In [3]:
# Import the file, using injection date, age groups, and sex as index columns
df = pd.read_csv("vaccination-rates-over-time-by-age.csv", index_col=[0, 1, 2])
# Drop the age groups, and sex columns
df = df.droplevel([1, 2])
# Sum up all numbers in the same injection date
df = df.groupby(pd.to_datetime(df.index, dayfirst=True).date).sum()
# Sum up the doses of the respective vaccines type
df_total = pd.concat([df.iloc[:, :3].sum(axis=1), df.iloc[:, 3:].sum(axis=1)], axis=1)
df_total.columns = ["Sinovac", "BioNTech"]
# Get total sum of all dates
df_total = df_total.cumsum()
# Get total sum of all vaccines
df_total["Total"] = df_total.sum(axis=1)
print("Total:")
display(df_total.iloc[[-1]])
print("Total Percentage:")
display(df_total.iloc[[-1]]/df_total.iloc[-1, -1])

# Get first dose of the respective vaccines type
df_first = df[["Sinovac 1st dose", "BioNTech 1st dose"]]
# Get total sum of all dates
df_first = df_first.cumsum()
# Get total sum of all vaccines
df_first["Total"] = df_first.sum(axis=1)
print("First dose:")
display(df_first.iloc[[-1]])
print("First dose Percentage:")
display(df_first.iloc[[-1]]/df_first.iloc[-1, -1])

# VR
print(f"VR: {df_first.iloc[-1, -1]/total_population}")

Total:


,Sinovac,BioNTech,Total
2022-01-23,4128952,6661061,10790013


Total Percentage:


,Sinovac,BioNTech,Total
2022-01-23,0.382664,0.617336,1.0


First dose:


,Sinovac 1st dose,BioNTech 1st dose,Total
2022-01-23,2007442,3238645,5246087


First dose Percentage:


,Sinovac 1st dose,BioNTech 1st dose,Total
2022-01-23,0.382655,0.617345,1.0


VR: 0.7789752917767945


In [115]:
# Import the file, using injection date, age groups, and sex as index columns
df = pd.read_csv("vaccination-rates-over-time-by-age.csv", index_col=[0, 1, 2])
df = df.iloc[:, [0, 3]]
# Drop the sex columns and merge DF
df = df.droplevel([2])
df = df.groupby(df.index).sum()
df.index = pd.MultiIndex.from_tuples([(datetime.strptime(x[0], "%d/%m/%Y"), x[1]) for x in df.index])
# Get the latest date data
df = df.loc[sorted(df.index)[-1][0]]
# Get perecntage
df = df*100/df.sum(axis=0)

# Plot
df = df.reset_index()
df['index'][0] = "12-19"
fig = px.bar(df, x="index", y=["Sinovac 1st dose", "BioNTech 1st dose"], title="Distribution of 1st dose Vaccine Type by Age Group", barmode="group")
fig.update_layout(titlefont_size=14,
            xaxis_titlefont_size=10,
            xaxis_tickfont_size=10,
            yaxis_titlefont_size=10,
            yaxis_tickfont_size=10,
            legend_font_size=10,
            width=720, 
            height=480)
fig.update_yaxes(title_text="%")
fig.show()
fig.write_image("images/dist_vaccine.jpg", width=720, height=480)

<ipython-input-115-eefa50f3ded2>:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [160]:
df = pd.read_csv("vaccination-rates-over-time-by-age.csv", index_col=[0, 1, 2])
# Sum up the vaccination number of male and female
df = df.groupby(["Date", "Age Group"]).sum()
# Get the first dose data, and sparse out columns by age group
df = (df.iloc[:, 0] + df.iloc[:, 3].values).unstack("Age Group").fillna(0)
# Convert the indexes into DateTime objects
df.index = pd.to_datetime(df.index, dayfirst=True)
df_ = df
# Resample to weekly resolution
df = df.resample("w").sum()
# Age groups total population
df.columns = ["12-19", "20-29", "30-39", "40-49", "50-59", "60-69", "70-79", "80 and above"]

# Reindex for plotting
df.index.name = "time"
df = df.reset_index()

fig = make_subplots(rows=1, cols=1)
# Get the absolute difference of vaccination rate (gradient)
for i in range(1, len(df.columns)):
    fig.add_trace(go.Scatter(x=df["time"], y=df[df.columns[i]].diff().fillna(0), name=df.columns[i]), row=1, col=1)
fig.update_layout(title_text='<span style="font-size: 14px;">Weekly Change in Newly Vaccinated Population of COVID-19 Vaccine in Hong Kong by Age Group</span>',
                  xaxis_title="Time",
                  legend_title="Age",
                  xaxis=dict(
                      dtick = "M1",
                      showgrid = False
                  ),
                  xaxis_titlefont_size=10,
                  xaxis_tickfont_size=10,
                  yaxis_titlefont_size=10,
                  yaxis_tickfont_size=10,
                  legend_font_size=10,
                  width=720, height=480)
fig.update_yaxes(title_text="#", showgrid=True)
fig.show()
fig.write_image("images/chg_no_inject.jpg", width=720, height=480)

### Vaccination Rate by age groups

In [143]:
df = pd.read_csv("vaccination-rates-over-time-by-age.csv", index_col=[0, 1, 2])
# Sum up the vaccination number of male and female
df = df.groupby(["Date", "Age Group"]).sum()
# Get the first dose data, and sparse out columns by age group
df = (df.iloc[:, 0] + df.iloc[:, 3].values).unstack("Age Group").fillna(0)
# Convert the indexes into DateTime objects
df.index = pd.to_datetime(df.index, dayfirst=True)
df_ = df
# Resample to weekly resolution
df = df.resample("w").sum()

# Age groups total population
df.columns = ["12-19", "20-29", "30-39", "40-49", "50-59", "60-69", "70-79", "80 and above"]
pop = pd.Series([448800, 808700, 1126300, 1142500, 1174200, 1071800, 560500, 401800], index=df.columns)

# Calculate the vaccination rate
df = df.cumsum() * 100 / pop
# Print the latest vaccination rate
display(df.iloc[[-1]])
# Reindex for plotting
df.index.name = "time"
df = df.reset_index()

# Get infected numbers
df2 = pd.read_excel("previous_cases_covid19_en.xlsx", usecols=[1])
df2["#"] = 1
df2 = df2.groupby("Report date").sum()
df2 = df2.loc["2021-2-22":]

,12-19,20-29,30-39,40-49,50-59,60-69,70-79,80 and above
Date,,,,,,,,
2022-01-23,83.007799,82.813157,83.985261,90.627921,85.408959,72.798097,57.496699,29.143106


In [158]:
fig = make_subplots(specs=[[{"secondary_y": True}]], rows=1, cols=1)
for i in range(1, len(df.columns)):
    fig.add_trace(go.Scatter(x=df["time"], y=df[df.columns[i]], name=df.columns[i]), row=1, col=1)
fig.add_bar(x=df2.index, y=df2["#"], secondary_y=True, name="Confirmed case #", marker={"color":"brown"})
fig.update_layout(title_text='<span style="font-size: 14px;">Vaccination Rate of COVID-19 Vaccine in Hong Kong by Age Group</span>',
                  xaxis_title="Time",
                  legend_title="Age",
                  xaxis=dict(
                      dtick = "M1",
                      showgrid = False,
                  ),
                  xaxis_titlefont_size=10,
                  xaxis_tickfont_size=10,
                  yaxis_titlefont_size=10,
                  yaxis_tickfont_size=10,
                  legend_font_size=10,
                  width=800, height=480)
fig.update_yaxes(title_text="%", showgrid=True, range=[-20, 100], secondary_y=False)
fig.update_yaxes(title_text="#", showgrid=False, range=[0, 150], secondary_y=True,
                 titlefont_size=10,
                 tickfont_size=10)
fig.show()

In [159]:
fig.write_image("images/vr_age.jpg", width=800, height=480)

### Gradient of vaccination rate across age groups

In [108]:
fig = make_subplots(rows=1, cols=1)
# Get the absolute difference of vaccination rate (gradient)
for i in range(1, len(df.columns)):
    fig.add_trace(go.Scatter(x=df["time"], y=df[df.columns[i]].diff().fillna(0), name=df.columns[i]), row=1, col=1)
fig.update_layout(title_text='<span style="font-size: 14px;">Absolute Change in Weekly Vaccination Rate of COVID-19 Vaccine in Hong Kong by Age Group</span>',
                  xaxis_title="Time",
                  legend_title="Age",
                  xaxis=dict(
                      dtick = "M1",
                      showgrid = False
                  ),
                  xaxis_titlefont_size=10,
                  xaxis_tickfont_size=10,
                  yaxis_titlefont_size=10,
                  yaxis_tickfont_size=10,
                  legend_font_size=10,
                  width=720, height=480)
fig.update_yaxes(title_text="%", showgrid=True)
fig.show()

In [109]:
fig.write_image("images/abschg_vr_age.jpg", width=720, height=480)

### Acceleration of vaccination rate across age groups

In [25]:
# Get second differencing of the absolute difference of vaccination rate (AWAVR)
dfx = df[[x for x in df.columns if x != "time"]].diff().diff()

dfx.index = df['time']
display(dfx)
dfx.to_csv("Diff_weeklyAbsChg.csv")

,12-19,20-29,30-39,40-49,50-59,60-69,70-79,80 and above
time,,,,,,,,
2021-02-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-03-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-03-14,0.088904,0.423767,0.578265,0.762013,0.660365,0.196119,-0.374309,-0.352912
2021-03-21,0.178030,0.501917,1.782118,2.051904,1.753619,-0.152454,-0.168956,-0.094823
2021-03-28,-0.190062,-0.520218,-1.158484,-1.471772,-1.298586,-1.553648,-1.088849,-0.325535
2021-04-04,-0.086898,-0.300977,-0.920891,-1.258556,-1.066088,-0.962773,-0.602855,-0.174963
2021-04-11,0.236185,0.487944,1.724940,1.852254,1.118293,0.428438,0.259233,0.078646
2021-04-18,0.014483,0.393224,0.894433,0.954923,0.681145,0.588076,0.354683,0.094823
2021-04-25,-0.011364,0.082354,-0.461955,-0.560000,-0.367058,-0.182497,-0.135950,-0.042061


### Statistical test
We use dummy variable of the policy's presence (0/1) as independent (catagorical) variable; AWAVR as dependent (continuous) variable. We'll check normality by Shapiro-Wilk test (alpha at 0.05), then decide using Mann-Whitney U test (non-normally distributed) or Independent T-test (normally distributed) for covariate testing, both alphas at 0.05.

For policies, please refer to table 1 of the paper.

#### AWAVR in "Eligibility".
This is only targetted on age group 0-19. 

In [28]:
from scipy.stats import ttest_ind, mannwhitneyu, shapiro
import numpy as np

# Construct data and dummy variable column
awavr_0_19 = dfx.iloc[:, [0]]
awavr_0_19 = (awavr_0_19 - awavr_0_19.mean()) / awavr_0_19.std()
awavr_0_19["policy"] = 0
i = awavr_0_19.index.get_loc("2021-04-11")
awavr_0_19.iloc[i:i+4, 1] = 1
print(f"AWAVR 0-19 2021-04-11: mean {np.mean(awavr_0_19.iloc[i:i+4, 0]):.2f} std {np.std(awavr_0_19.iloc[i:i+4, 0]):.2f}")
i = awavr_0_19.index.get_loc("2021-09-12")
awavr_0_19.iloc[i:i+4, 1] = 1
print(f"AWAVR 0-19 2021-09-12: mean {np.mean(awavr_0_19.iloc[i:i+4, 0]):.2f} std {np.std(awavr_0_19.iloc[i:i+4, 0]):.2f}")
awavr_0_19 = awavr_0_19.dropna()

# Shapiro-Wilk test for normality.
norm = shapiro(awavr_0_19["12-19"])
print(f"The p-value of Shapiro-Wilk test: {norm.pvalue}, indicating it is {'not ' if norm.pvalue < 0.05 else ''}normally distributed.")

# Run statistical test based on normality check.
if norm.pvalue > 0.05:
    result = ttest_ind(awavr_0_19["12-19"], awavr_0_19["policy"])
    print(f"The p-value of Independent T-test: {result.pvalue}.")
else:
    result = mannwhitneyu(awavr_0_19["12-19"], awavr_0_19["policy"])
    print(f"The p-value of Mann-Whitney U test: {result.pvalue}.")

AWAVR 0-19 2021-04-11: mean 0.23 std 0.35
AWAVR 0-19 2021-09-12: mean -1.46 std 1.29
The p-value of Shapiro-Wilk test: 0.006368536036461592, indicating it is not normally distributed.
The p-value of Mann-Whitney U test: 0.12248376667975036.


#### AWAVR in "Accesibility".

In [29]:
# Construct data and dummy variable column
awavr_0_19 = dfx.iloc[:, [0]]
awavr_0_19.columns = ["AWAVR"]
awavr_0_19 = (awavr_0_19 - awavr_0_19.mean()) / awavr_0_19.std()
awavr_0_19["policy"] = 0
i = awavr_0_19.index.get_loc("2021-06-20")
awavr_0_19.iloc[i:i+4, 1] = 1
print(f"AWAVR 0-19 2021-06-20: mean {np.mean(awavr_0_19.iloc[i:i+4, 0]):.2f} std {np.std(awavr_0_19.iloc[i:i+4, 0]):.2f}")
i = awavr_0_19.index.get_loc("2021-06-27")
awavr_0_19.iloc[i:i+4, 1] = 1
print(f"AWAVR 0-19 2021-06-27: mean {np.mean(awavr_0_19.iloc[i:i+4, 0]):.2f} std {np.std(awavr_0_19.iloc[i:i+4, 0]):.2f}")

awavr_60up = dfx.iloc[:, -3:] @ np.array([1071800, 560500, 401800]).reshape(-1, 1)
awavr_60up.columns = ["AWAVR"]
awavr_60up = (awavr_60up - awavr_60up.mean()) / awavr_60up.std()
awavr_60up["policy"] = 0
i = awavr_60up.index.get_loc("2021-04-11")
awavr_60up.iloc[i:i+4, 1] = 1
print(f"AWAVR 60up 2021-04-11: mean {np.mean(awavr_60up.iloc[i:i+4, 0]):.2f} std {np.std(awavr_60up.iloc[i:i+4, 0]):.2f}")

awavr_70up = dfx.iloc[:, -2:] @ np.array([560500, 401800]).reshape(-1, 1)
awavr_70up.columns = ["AWAVR"]
awavr_70up = (awavr_70up - awavr_70up.mean()) / awavr_70up.std()
awavr_70up["policy"] = 0
i = awavr_70up.index.get_loc("2021-07-25")
awavr_70up.iloc[i:i+4, 1] = 1
print(f"AWAVR 70up 2021-07-25: mean {np.mean(awavr_60up.iloc[i:i+4, 0]):.2f} std {np.std(awavr_60up.iloc[i:i+4, 0]):.2f}")

awavr = pd.concat([awavr_0_19, awavr_60up, awavr_70up])
awavr = awavr.dropna()

# Shapiro-Wilk test for normality.
norm = shapiro(awavr["AWAVR"])
print(f"The p-value of Shapiro-Wilk test: {norm.pvalue}, indicating it is {'not' if norm.pvalue < 0.05 else ''} normally distributed.")

# Run statistical test based on normality check.
if norm.pvalue > 0.05:
    result = ttest_ind(awavr["AWAVR"], awavr["policy"])
    print(f"The p-value of Independent T-test: {result.pvalue}.")
else:
    result = mannwhitneyu(awavr["AWAVR"], awavr["policy"])
    print(f"The p-value of Mann-Whitney U test: {result.pvalue}.")

AWAVR 0-19 2021-06-20: mean 1.51 std 0.99
AWAVR 0-19 2021-06-27: mean 1.16 std 0.59
AWAVR 60up 2021-04-11: mean 0.18 std 0.63
AWAVR 70up 2021-07-25: mean 0.37 std 0.39
The p-value of Shapiro-Wilk test: 1.7574529071612277e-12, indicating it is not normally distributed.
The p-value of Mann-Whitney U test: 0.0006543241732797742.


#### AWAVR in "Incentive (reward)".

In [30]:
# Construct data and dummy variable column
awavr_0_19 = dfx.iloc[:, [0]]
awavr_0_19.columns = ["AWAVR"]
awavr_0_19 = (awavr_0_19 - awavr_0_19.mean()) / awavr_0_19.std()
awavr_0_19["policy"] = 0
i = awavr_0_19.index.get_loc("2021-08-01")
awavr_0_19.iloc[i:i+4, 1] = 1
print(f"AWAVR 0-19 2021-08-01: mean {np.mean(awavr_60up.iloc[i:i+4, 0]):.2f} std {np.std(awavr_60up.iloc[i:i+4, 0]):.2f}")

awavr_all = dfx @ np.array([448800, 808700, 1126300, 1142500, 1174200, 1071800, 560500, 401800]).reshape(-1, 1)
awavr_all.columns = ["AWAVR"]
awavr_all = (awavr_all - awavr_all.mean()) / awavr_all.std()
awavr_all["policy"] = 0
i = awavr_all.index.get_loc("2021-05-30")
awavr_all.iloc[i:i+4, 1] = 1
print(f"AWAVR all 2021-05-30: mean {np.mean(awavr_60up.iloc[i:i+4, 0]):.2f} std {np.std(awavr_60up.iloc[i:i+4, 0]):.2f}")

awavr_20_69 = dfx.iloc[:, 1:-2] @ np.array([808700, 1126300, 1142500, 1174200, 1071800]).reshape(-1, 1)
awavr_20_69.columns = ["AWAVR"]
awavr_20_69 = (awavr_20_69 - awavr_20_69.mean()) / awavr_20_69.std()
awavr_20_69["policy"] = 0
i = awavr_20_69.index.get_loc("2021-05-30")
awavr_20_69.iloc[i:i+4, 1] = 1
print(f"AWAVR 20-69 2021-05-30: mean {np.mean(awavr_60up.iloc[i:i+4, 0]):.2f} std {np.std(awavr_60up.iloc[i:i+4, 0]):.2f}")

awavr = pd.concat([awavr_0_19, awavr_all, awavr_20_69])
awavr = awavr.dropna()

# Shapiro-Wilk test for normality.
norm = shapiro(awavr["AWAVR"])
print(f"The p-value of Shapiro-Wilk test: {norm.pvalue}, indicating it is {'not' if norm.pvalue < 0.05 else ''} normally distributed.")

# Run statistical test based on normality check.
if norm.pvalue > 0.05:
    result = ttest_ind(awavr["AWAVR"], awavr["policy"])
    print(f"The p-value of Independent T-test: {result.pvalue}.")
else:
    result = mannwhitneyu(awavr["AWAVR"], awavr["policy"])
    print(f"The p-value of Mann-Whitney U test: {result.pvalue}.")

AWAVR 0-19 2021-08-01: mean 0.11 std 0.79
AWAVR all 2021-05-30: mean 0.68 std 0.56
AWAVR 20-69 2021-05-30: mean 0.68 std 0.56
The p-value of Shapiro-Wilk test: 5.957730991212884e-06, indicating it is not normally distributed.
The p-value of Mann-Whitney U test: 0.005115539479896202.


#### AWAVR in "Restriction".

In [31]:
# Construct data and dummy variable column
awavr_all = dfx @ np.array([448800, 808700, 1126300, 1142500, 1174200, 1071800, 560500, 401800]).reshape(-1, 1)
awavr_all.columns = ["AWAVR"]
awavr_all = (awavr_all - awavr_all.mean()) / awavr_all.std()
awavr_all["policy"] = 0
i = awavr_all.index.get_loc("2021-12-26")
awavr_all.iloc[i:i+4, 1] = 1
print(f"AWAVR all 2021-12-26: mean {np.mean(awavr_60up.iloc[i:i+4, 0]):.2f} std {np.std(awavr_60up.iloc[i:i+4, 0]):.2f}")

awavr_20_69 = dfx.iloc[:, 1:-2] @ np.array([808700, 1126300, 1142500, 1174200, 1071800]).reshape(-1, 1)
awavr_20_69.columns = ["AWAVR"]
awavr_20_69 = (awavr_20_69 - awavr_20_69.mean()) / awavr_20_69.std()
awavr_20_69["policy"] = 0
i = awavr_20_69.index.get_loc("2021-08-01")
awavr_20_69.iloc[i:i+4, 1] = 1
print(f"AWAVR 20-69 2021-08-01: mean {np.mean(awavr_60up.iloc[i:i+4, 0]):.2f} std {np.std(awavr_60up.iloc[i:i+4, 0]):.2f}")
i = awavr_20_69.index.get_loc("2021-11-07")
awavr_20_69.iloc[i:i+4, 1] = 1
print(f"AWAVR 20-69 2021-11-07: mean {np.mean(awavr_60up.iloc[i:i+4, 0]):.2f} std {np.std(awavr_60up.iloc[i:i+4, 0]):.2f}")
i = awavr_20_69.index.get_loc("2021-12-19")
awavr_20_69.iloc[i:i+4, 1] = 1
print(f"AWAVR 20-69 2021-12-19: mean {np.mean(awavr_60up.iloc[i:i+4, 0]):.2f} std {np.std(awavr_60up.iloc[i:i+4, 0]):.2f}")

awavr = pd.concat([awavr_all, awavr_20_69])
awavr = awavr.dropna()

# Shapiro-Wilk test for normality.
norm = shapiro(awavr["AWAVR"])
print(f"The p-value of Shapiro-Wilk test: {norm.pvalue}, indicating it is {'not' if norm.pvalue < 0.05 else ''} normally distributed.")

# Run statistical test based on normality check.
if norm.pvalue > 0.05:
    result = ttest_ind(awavr["AWAVR"], awavr["policy"])
    print(f"The p-value of Independent T-test: {result.pvalue}.")
else:
    result = mannwhitneyu(awavr["AWAVR"], awavr["policy"])
    print(f"The p-value of Mann-Whitney U test: {result.pvalue}.")

AWAVR all 2021-12-26: mean 1.25 std 2.12
AWAVR 20-69 2021-08-01: mean 0.11 std 0.79
AWAVR 20-69 2021-11-07: mean 0.02 std 0.10
AWAVR 20-69 2021-12-19: mean 1.21 std 2.15
The p-value of Shapiro-Wilk test: 4.8435336793772876e-05, indicating it is not normally distributed.
The p-value of Mann-Whitney U test: 0.0005540482878599383.
